### 가장 기초적인 방식의 Two-Layer Net 구현

In [2]:
import torch

device = torch.device('cpu') # ('cuda:0') for GPU option

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
w1 = torch.randn(D_in, H, device=device)
w2 = torch.randn(H, D_out, device=device)

learning_rate = 1e-6
for t in range(500):
    # Forward Pass
    h = x.mm(w1) # matrix multiplication. 첫번째 레이어 계산
    h_relu = h.clamp(min=0) # ReLU 활성함수 적용
    y_pred = h_relu.mm(w2) # 2번째 레이어 계산
    loss = (y_pred - y).pow(2).sum() # Loss 제곱합
    
    # Back propagation
    # gradient 계산(미분 연쇄법칙)
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred) # .t() : Transpose
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

### Pytorch Autograd 기능을 사용한 방식
- loss.backward()를 사용하여 연쇄적인 gradient계산

In [4]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
w1 = torch.randn(D_in, H, requires_grad=True) # 가중치 행렬임을 표시. gradient 계산을 시행할 대상
w2 = torch.randn(H, D_out, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    
    # w1, w2에 대해서 gradient of loss를 계산한다.
    loss.backward()
    
    with torch.no_grad(): # 현 시점에서의 동작은 gradient 계산에서 제외
        # gradient step 실행
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_() # gradient 초기화(다음 step에서 다시 계산할 것이므로)
        w2.grad.zero_()

### New Autograd function 사용하기

In [8]:
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_y):
        x, = ctx.saved_tensors
        grad_input = grad_y.clone()
        grad_input[x < 0] = 0
        return grad_input
    
def my_relu(x):
    return MyReLU.apply(x)

In [10]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
w1 = torch.randn(D_in, H, requires_grad=True)
w2 = torch.randn(H, D_out, requires_grad=True)

learning_rate = 1e-6

# 사실 현재 케이스에서는 새로운 autograd function을 정의할 필요는 업다
# custom backward가 필요한 경우에만 이를 활용하므로, 현 케이스에서는 그냥 일반적인 function으로 Forward pass를 진행한다.
def my_relu(x):
    return x.clamp(min=0)

for t in range(500):
    y_pred = my_relu(x.mm(w1)).mm(w2)  # 새로운 autograd 적용 function으로 forward pass 시 적용
    loss = (y_pred - y).pow(2).sum()
    
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

### torch.nn : Neural Net의 high level wrapper 활용

In [11]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Pytorch 프레임워크에서 제공하는 Layer 배치 기능
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H), # Fully Connected Layer
    torch.nn.ReLU(), # 각 레이어는 학습 가능한 Weight를 지니고 있다
    torch.nn.Linear(H, D_out),
)

learning_rate = 1e-2

for t in range(500):
    y_pred = model(x) # Forward pass를 사전에 정의해 둔 모델 동작으로 간소화 가능
    loss = torch.nn.functional.mse_loss(y_pred, y) # Forward pass를 통과한 예측값에 대해 MSE Loss 적용
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters(): # 모델의 모든 파라미터에 대하여 적용
            param -= learning_rate * param.grad
    model.zero_grad() # 전체 모델에서 사용된 가중치 기울기 초기화

### Optimizer의 사용

In [13]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

learning_rate = 1e-4
# gradient step 과정을 대신하는 기능. 모델의 학습 가능한 파라미터에 대해서, 학습율에 기반해 step 적용.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # optimizer에 따라서 learning rate 조정이 적용된다

for t in range(500):
    y_pred = model(x)
    loss = torch.nn.functional.mse_loss(y_pred, y)
    
    loss.backward()
    
    # 앞서 모델에서 사용된 모든 파라미터들에 대해 gradient step을 적용하는 과정을 optimizer를 통해 간소화
    optimizer.step()
    optimizer.zero_grad()

### Define new NN Module

In [14]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    y_pred = model(x)
    loss = torch.nn.functional.mse_loss(y_pred, y)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

### DataLoader
- 데이터셋을 다루기 위한 클래스로, batching, shuffling, multithreding 기능을 제공한다.
- 새로운 custom data를 사용할 때 Dataset class를 새롭게 작성하면 된다.

In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

loader = DataLoader(TensorDataset(x, y), batch_size=8) # 8개 단위의 미니배치 사용. TensorDataset 사용할 것.
model = TwoLayerNet(D_in, H, D_out)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
for epoch in range(20):
    for x_batch, y_batch in loader:
        y_pred = model(x_batch) # 매 반복마다 미니배치가 계속해서 로드된다
        loss = torch.nn.functional.mse_loss(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

### Pretrained Model 사용
- torchvision을 통해서 기존의 pretrained 된 model을 사용할 수 있게 해준다.

In [16]:
import torch
import torchvision

alexnet = torchvision.models.alexnet(pretrained=True)
vgg16 = torchvision.models.vgg16(pretrained=True)
resnet101 = torchvision.models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/hanoul/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/hanoul/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

KeyboardInterrupt: 

### ONNX
- 뉴럴 네트워크 모델의 스탠다드한 형태
- 오픈소스로 하나의 프레임워크에서 학습된 모델을 여러 딥러닝 프레임워크에서 공용으로 사용될 수 있는 형태
- Pytorch는 기본적으로 Dynamic Graph로 학습을 진행하기 때문에, Static한 형태로 변환하여 사용하기 위해서는 ONNX를 사용해 Export해야 한다.
(ONNX 변환은 Caffe2로 export하여 Production, mobile 환경에서 사용하기에 편리)

In [ ]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

dummy_input = torch.randn(N, D_in)
torch.onnx.export(model, dummy_input, 'model.proto', verbose=True)